In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/records500.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: records500/19000/19339_hr.dat  
  inflating: records500/19000/19339_hr.hea  
  inflating: records500/19000/19340_hr.dat  
  inflating: records500/19000/19340_hr.hea  
  inflating: records500/19000/19341_hr.dat  
  inflating: records500/19000/19341_hr.hea  
  inflating: records500/19000/19342_hr.dat  
  inflating: records500/19000/19342_hr.hea  
  inflating: records500/19000/19343_hr.dat  
  inflating: records500/19000/19343_hr.hea  
  inflating: records500/19000/19344_hr.dat  
  inflating: records500/19000/19344_hr.hea  
  inflating: records500/19000/19345_hr.dat  
  inflating: records500/19000/19345_hr.hea  
  inflating: records500/19000/19346_hr.dat  
  inflating: records500/19000/19346_hr.hea  
  inflating: records500/19000/19347_hr.dat  
  inflating: records500/19000/19347_hr.hea  
  inflating: records500/19000/19348_hr.dat  
  inflating: records500/19000/19348_hr.hea  
  inflating: records500/19000/19

In [9]:
import pandas as pd
import numpy as np
!pip install wfdb
import wfdb
import warnings; warnings.filterwarnings('ignore')
import ast
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = '/content/'
sampling_rate=500

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))


# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return ', '.join(list(set(tmp)))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 4.0 MB/s eta 0:00:00


In [10]:
# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

In [11]:
#Суммируем отведения из одного сэмпла(1-ые с 1-ым, 2-ые со 2-ым и т.д.), чтобы превратить трехмерный массив в двумерный
X = np.sum(X, axis=1)
X = pd.DataFrame(X)
X

,0,1,2,3,4,5,6,7,8,9,10,11
0,13.047,11.561,-1.486,-12.346,7.539,5.145,-5.719,44.027,-1.304,-15.227,4.821,12.159
1,-17.863,-17.160,0.734,17.529,-9.284,-8.311,22.419,63.058,60.372,4.753,15.615,1.959
2,-13.900,28.343,42.218,-7.439,-28.050,35.170,-1.493,2.635,-15.551,1.491,25.979,21.763
3,-5.312,85.469,90.745,-40.140,-48.043,88.007,-18.092,-59.622,22.646,113.973,69.196,53.913
4,-17.993,-12.427,5.521,15.232,-11.631,-3.372,7.634,0.434,-1.649,10.435,0.325,0.212
...,...,...,...,...,...,...,...,...,...,...,...,...
21794,-57.826,-29.689,28.124,43.314,-42.474,-0.954,-1.213,-14.125,23.645,-13.371,-20.514,-22.209
21795,-15.430,-20.143,-4.727,17.623,-4.856,-12.121,-7.535,-14.867,-12.754,-11.040,-20.255,-16.447
21796,-8.500,-26.027,-17.516,17.470,5.038,-21.156,12.565,2.085,3.849,0.061,-10.200,-5.451
21797,-5.592,-18.072,-12.464,12.100,3.991,-14.819,-8.844,-5.369,-37.957,-10.578,-3.504,1.046


In [12]:
#Создаем столбец с диагнозом здоров/нездоров
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
Y['Condition'] = Y['diagnostic_superclass']
for i in range(len(Y['Condition'])):
    if Y['diagnostic_superclass'][i] == 'NORM':
        Y['Condition'][i] = 'Healthy'
    else:
        Y['Condition'][i] = 'Unhealthy'

In [13]:
#Соединяем нужные колонки без пропусков из ptbxl_database.csv('age', 'sex') с данными ЭКГ в один датафрэйм из 14 столбцов
df = Y[['age', 'sex']]
X_df = pd.concat([X, df], axis=1)
X_df.columns = X_df.columns.astype(str)
X_df

,0,1,2,3,4,5,6,7,8,9,10,11,age,sex
0,13.047,11.561,-1.486,-12.346,7.539,5.145,-5.719,44.027,-1.304,-15.227,4.821,12.159,56.0,1
1,-17.863,-17.160,0.734,17.529,-9.284,-8.311,22.419,63.058,60.372,4.753,15.615,1.959,19.0,0
2,-13.900,28.343,42.218,-7.439,-28.050,35.170,-1.493,2.635,-15.551,1.491,25.979,21.763,37.0,1
3,-5.312,85.469,90.745,-40.140,-48.043,88.007,-18.092,-59.622,22.646,113.973,69.196,53.913,24.0,0
4,-17.993,-12.427,5.521,15.232,-11.631,-3.372,7.634,0.434,-1.649,10.435,0.325,0.212,19.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21794,-57.826,-29.689,28.124,43.314,-42.474,-0.954,-1.213,-14.125,23.645,-13.371,-20.514,-22.209,67.0,1
21795,-15.430,-20.143,-4.727,17.623,-4.856,-12.121,-7.535,-14.867,-12.754,-11.040,-20.255,-16.447,300.0,0
21796,-8.500,-26.027,-17.516,17.470,5.038,-21.156,12.565,2.085,3.849,0.061,-10.200,-5.451,59.0,1
21797,-5.592,-18.072,-12.464,12.100,3.991,-14.819,-8.844,-5.369,-37.957,-10.578,-3.504,1.046,64.0,1


In [14]:
from sklearn import datasets, svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MultiLabelBinarizer, MinMaxScaler, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score, roc_auc_score

In [28]:
#Таргет
y = Y['Condition']
y.head(10)

,Condition
0,Healthy
1,Healthy
2,Healthy
3,Healthy
4,Healthy
5,Healthy
6,Healthy
7,Unhealthy
8,Healthy
9,Healthy


In [29]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y = y_encoded

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)
std = StandardScaler()
std.fit(X_train)
X_train = std.transform(X_train)
X_test = std.transform(X_test)

In [65]:
#Baseline 0.7004587155963303
model = SVC(C=10, kernel='linear', shrinking=False, tol=1, class_weight='balanced')
model.fit(X_train, y_train)
pred = model.predict(X_test)
print('accuracy_score:', accuracy_score(y_test, pred))

accuracy_score: 0.7004587155963303


Модель мультиклассовой классификации методом опорных векторов

In [75]:
#Таргет
y = Y['diagnostic_superclass']
y.head(10)

,diagnostic_superclass
0,NORM
1,NORM
2,NORM
3,NORM
4,NORM
5,NORM
6,NORM
7,MI
8,NORM
9,NORM


In [76]:
#классы
Y['diagnostic_superclass'].value_counts()

,count
diagnostic_superclass,
NORM,9069
MI,2532
STTC,2400
CD,1708
"CD, MI",1297
"STTC, HYP",781
"STTC, MI",599
HYP,535
"STTC, CD",471


In [77]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y = y_encoded

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)
std = StandardScaler()
std.fit(X_train)
X_train = std.transform(X_train)
X_test = std.transform(X_test)

In [88]:
#Baseline 0.37545871559633026
model = LinearSVC(dual=False, C=0.001, multi_class='ovr', class_weight='balanced')
model.fit(X_train, y_train)
pred = model.predict(X_test)
print('accuracy_score:', accuracy_score(y_test, pred))

accuracy_score: 0.37545871559633026
